In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report,mean_squared_error, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedBaggingClassifier
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from statsmodels.discrete.discrete_model import MNLogit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

In [2]:
# put the dataset which comes from short_dataset_feature_engineering.ipynb 
file_path = '/data/caysar9/results/final_short.csv'
df = pd.read_csv(file_path)

In [3]:
# Define features and target 
features = ['trigger_lack_physical_activity','trigger_physical_activity','trigger_poor_sleep','trigger_stress','sleep_duration_hours','sleep_duration_past_7_days','age','migraine_attacks_past7days','mean_migraine_duration_past7days','total_physical_activity','reported_anxiety', 'reported_depression']


target = 'duration_in_hours'

X = df[features]
y = df[target]

# Define bins for migraine duration (Low: 4-9, Medium: 10-22, High: 23-72 hours)
bins = [4, 9, 22, 72]
labels = ['Low', 'Medium', 'High']  # Descriptive labels for Low, Medium, High
y_binned = pd.cut(y, bins=bins, labels=labels, include_lowest=True)

# Frequency counts
print("Frequency counts for each duration class before sampling:")
print(y_binned.value_counts())

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_binned, test_size=0.2, random_state=42, stratify=y_binned)

# Frequency counts before applying SMOTE
print("\nFrequency counts before SMOTE:")
print("Training set:")
print(y_train.value_counts())
print("Test set:")
print(y_test.value_counts())

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Frequency counts after applying SMOTE
print("\nFrequency counts after SMOTE:")
print(y_train_balanced.value_counts())


Frequency counts for each duration class before sampling:
duration_in_hours
Low       7409
Medium    6842
High       151
Name: count, dtype: int64

Frequency counts before SMOTE:
Training set:
duration_in_hours
Low       5927
Medium    5473
High       121
Name: count, dtype: int64
Test set:
duration_in_hours
Low       1482
Medium    1369
High        30
Name: count, dtype: int64

Frequency counts after SMOTE:
duration_in_hours
Low       5927
Medium    5927
High      5927
Name: count, dtype: int64


In [13]:
# Define hyperparameter distributions
param_distributions = {
    'C': uniform(0.01, 10),  # Regularization strength
    'max_iter': randint(100, 1000),  # Number of iterations
    'solver': ['lbfgs', 'newton-cg', 'sag'],  # Solvers that support multinomial
    'multi_class': ['multinomial']  # Multinomial setting for multi-class classification
}

# Initialize the Logistic Regression model
lr_model = LogisticRegression(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=lr_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Optimize based on AUC (One-vs-One for multi-class)
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions and evaluate
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))


Fitting 3 folds for each of 20 candidates, totalling 60 fits


/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/data/caysar9/venv-py311

Best Hyperparameters: {'C': 6.184815096277165, 'max_iter': 413, 'multi_class': 'multinomial', 'solver': 'newton-cg'}


/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



AUC Score (One-vs-One): 0.9178

Confusion Matrix:
[[  22    0    8]
 [  26 1308  148]
 [ 176  225  968]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.10      0.73      0.17        30
      Medium       0.85      0.88      0.87      1482
        High       0.86      0.71      0.78      1369

    accuracy                           0.80      2881
   macro avg       0.60      0.77      0.61      2881
weighted avg       0.85      0.80      0.82      2881



In [14]:
# Add a constant to the feature set for the intercept term
X_train_balanced_const = sm.add_constant(X_train_balanced)


# Fit a multinomial logistic regression model using statsmodels
logit_model = sm.MNLogit(y_train_balanced, X_train_balanced_const)
logit_results = logit_model.fit()

# Get the summary, including coefficients and p-values
print(logit_results.summary())

Optimization terminated successfully.
         Current function value: 0.416807
         Iterations 9
                          MNLogit Regression Results                          
Dep. Variable:      duration_in_hours   No. Observations:                17781
Model:                        MNLogit   Df Residuals:                    17755
Method:                           MLE   Df Model:                           24
Date:                Sat, 07 Dec 2024   Pseudo R-squ.:                  0.6206
Time:                        18:55:27   Log-Likelihood:                -7411.3
converged:                       True   LL-Null:                       -19534.
Covariance Type:            nonrobust   LLR p-value:                     0.000
        duration_in_hours=Medium       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -7.8481      0.218    -36.04

In [15]:
# Define hyperparameter distributions
param_distributions = {
    'criterion': ['gini', 'entropy', 'log_loss'],  # Splitting criteria
    'max_depth': randint(3, 50),  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 10),  # Minimum number of samples in a leaf node
    'max_features': [None, 'sqrt', 'log2']  # Number of features considered for the best split
}

# Initialize the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dt_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Evaluate based on AUC for multi-class problems
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions and evaluate
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Hyperparameters: {'criterion': 'log_loss', 'max_depth': 9, 'max_features': None, 'min_samples_leaf': 8, 'min_samples_split': 16}

AUC Score (One-vs-One): 0.9471

Confusion Matrix:
[[  23    0    7]
 [  17 1276  189]
 [ 111  169 1089]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.15      0.77      0.25        30
      Medium       0.88      0.86      0.87      1482
        High       0.85      0.80      0.82      1369

    accuracy                           0.83      2881
   macro avg       0.63      0.81      0.65      2881
weighted avg       0.86      0.83      0.84      2881



In [16]:
# Define hyperparameter distributions
param_distributions = {
    'n_estimators': randint(50, 300),  # Number of trees in the forest
    'max_depth': randint(5, 50),  # Maximum depth of each tree
    'min_samples_split': randint(2, 20),  # Minimum samples to split a node
    'min_samples_leaf': randint(1, 10),  # Minimum samples in a leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider for the best split
    'bootstrap': [True, False]  # Whether bootstrap samples are used
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Evaluate based on AUC for multi-class classification
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Hyperparameters: {'bootstrap': False, 'max_depth': 34, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 253}

AUC Score (One-vs-One): 0.9456

Confusion Matrix:
[[  18    0   12]
 [   2 1271  209]
 [  13  180 1176]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.55      0.60      0.57        30
      Medium       0.88      0.86      0.87      1482
        High       0.84      0.86      0.85      1369

    accuracy                           0.86      2881
   macro avg       0.75      0.77      0.76      2881
weighted avg       0.86      0.86      0.86      2881



In [17]:
# Get feature importance
feature_importance = best_model.feature_importances_
feature_names = X_train_balanced.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:\n", importance_df)



Feature Importance:
                              Feature  Importance
8   mean_migraine_duration_past7days    0.638196
7         migraine_attacks_past7days    0.116861
9            total_physical_activity    0.046713
6                                age    0.046062
5         sleep_duration_past_7_days    0.045474
4               sleep_duration_hours    0.042734
3                     trigger_stress    0.020322
2                 trigger_poor_sleep    0.019292
10                  reported_anxiety    0.010634
11               reported_depression    0.007725
0     trigger_lack_physical_activity    0.004765
1          trigger_physical_activity    0.001224


In [18]:
# Define hyperparameter distributions
param_distributions = {
    'n_estimators': randint(50, 300),  # Number of boosting stages
    'learning_rate': uniform(0.01, 0.3),  # Learning rate for boosting
    'max_depth': randint(3, 20),  # Maximum depth of each tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': randint(1, 10),  # Minimum number of samples required in a leaf node
    'subsample': uniform(0.7, 0.3),  # Fraction of samples used for fitting each base learner
    'max_features': ['sqrt', 'log2', None]  # Number of features to consider for splits
}

# Initialize the Gradient Boosting model
gb_model = GradientBoostingClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Evaluate based on AUC for multi-class classification
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions and evaluate
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))


Fitting 3 folds for each of 20 candidates, totalling 60 fits


Best Hyperparameters: {'learning_rate': 0.06597101766581075, 'max_depth': 13, 'max_features': 'log2', 'min_samples_leaf': 7, 'min_samples_split': 8, 'n_estimators': 239, 'subsample': 0.733015577358303}

AUC Score (One-vs-One): 0.9474

Confusion Matrix:
[[  18    0   12]
 [   1 1288  193]
 [   9  193 1167]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.64      0.60      0.62        30
      Medium       0.87      0.87      0.87      1482
        High       0.85      0.85      0.85      1369

    accuracy                           0.86      2881
   macro avg       0.79      0.77      0.78      2881
weighted avg       0.86      0.86      0.86      2881



In [19]:
# Get feature importance
feature_importance = best_model.feature_importances_
feature_names = X_train_balanced.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:\n", importance_df)



Feature Importance:
                              Feature  Importance
8   mean_migraine_duration_past7days    0.657411
7         migraine_attacks_past7days    0.116229
5         sleep_duration_past_7_days    0.046590
4               sleep_duration_hours    0.045678
6                                age    0.041423
9            total_physical_activity    0.040494
3                     trigger_stress    0.017699
2                 trigger_poor_sleep    0.016416
10                  reported_anxiety    0.008766
11               reported_depression    0.005562
0     trigger_lack_physical_activity    0.002885
1          trigger_physical_activity    0.000846


In [20]:
# Define hyperparameter distributions
param_distributions = {
    'n_neighbors': randint(1, 50),  # Number of neighbors
    'weights': ['uniform', 'distance'],  # Weighting strategies
    'metric': ['euclidean', 'manhattan', 'minkowski'],  # Distance metrics
    'p': randint(1, 3)  # distance parameter 
}

# Initialize the K-Nearest Neighbors model
knn_model = KNeighborsClassifier()

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=knn_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Evaluate based on AUC for multi-class classification
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions and evaluate
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Hyperparameters: {'metric': 'minkowski', 'n_neighbors': 22, 'p': 1, 'weights': 'distance'}

AUC Score (One-vs-One): 0.9193

Confusion Matrix:
[[  23    0    7]
 [  23 1282  177]
 [ 162  223  984]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.11      0.77      0.19        30
      Medium       0.85      0.87      0.86      1482
        High       0.84      0.72      0.78      1369

    accuracy                           0.79      2881
   macro avg       0.60      0.78      0.61      2881
weighted avg       0.84      0.79      0.81      2881



In [21]:
'''DNN:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
import numpy as np

# Define features and target
features = ['trigger_lack_physical_activity','trigger_physical_activity','trigger_poor_sleep','trigger_stress','sleep_duration_hours','sleep_duration_past_7_days','age','migraine_attacks_past7days','mean_migraine_duration_past7days','total_physical_activity','reported_anxiety', 'reported_depression']


target = 'duration_in_hours'

X = df[features]
y = df[target]

# Define bins for migraine duration (Low: 4-9, Medium: 10-22, High: 23-72 hours)
bins = [4, 9, 22, 72]
y_binned = np.digitize(y, bins=bins, right=True)
y_binned = np.clip(y_binned, 1, 3) 
y_encoded = y_binned - 1  # Map to [0, 1, 2]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Define a function to create the neural network model
def create_model(learning_rate=0.001, neurons=[64, 32, 16]):
    model = Sequential()
    model.add(Dense(neurons[0], activation='relu', input_shape=(X_train_scaled.shape[1],)))
    model.add(Dense(neurons[1], activation='relu'))
    model.add(Dense(neurons[2], activation='relu'))
    model.add(Dense(3, activation='softmax'))  # 3 classes: Low, Medium, High
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model for RandomizedSearchCV
keras_clf = KerasClassifier(model=create_model, verbose=0)

# Define hyperparameter distributions
param_distributions = {
    "model__learning_rate": [0.001, 0.01, 0.1],
    "model__neurons": [[64, 32, 16], [128, 64, 32], [32, 16, 8]],
    "batch_size": [16, 32, 64],
    "epochs": [20, 50, 100]
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=keras_clf,
    param_distributions=param_distributions,
    n_iter=10,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Optimize AUC for multi-class classification
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available cores
    random_state=42
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_scaled, y_train_smote)

# Get the best model and parameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Evaluate the model
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.predict_proba(X_test_scaled)

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
y_pred_classes = y_pred  
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=["Low", "Medium", "High"]))
'''

' \nfrom sklearn.metrics import roc_auc_score, confusion_matrix, classification_report\nfrom sklearn.model_selection import RandomizedSearchCV, train_test_split\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense\nfrom tensorflow.keras.optimizers import Adam\nfrom scikeras.wrappers import KerasClassifier\nfrom imblearn.over_sampling import SMOTE\nfrom sklearn.preprocessing import StandardScaler\nimport numpy as np\n\n# Define features and target\nfeatures = [\'trigger_lack_physical_activity\',\'trigger_physical_activity\',\'trigger_poor_sleep\',\'trigger_stress\',\'sleep_duration_hours\',\'sleep_duration_past_7_days\',\'age\',\'migraine_attacks_past7days\',\'mean_migraine_duration_past7days\',\'total_physical_activity\',\'reported_anxiety\', \'reported_depression\']\n\ntarget = \'attack_duration_hours\'\n\nX = df[features]\ny = df[target]\n\n# Define bins for migraine duration (Low: 4-9, Medium: 10-22, High: 23-72 hours)\nbins = [4, 9, 22, 72]\ny_